# 🤖 HackBot on Google Colab

Run HackBot entirely on Google Colab — no local CPU needed.
Supports **all three modes**: Chat, Agent, and Plan.

| Mode | Description |
|------|-------------|
| **Chat** | Interactive cybersecurity Q&A |
| **Agent** | Autonomous pen-testing with real tools |
| **Plan** | Structured penetration test plans |

---

## 1. Install HackBot

In [ ]:
# Install HackBot from GitHub
!pip install -q "hackbot @ git+https://github.com/yashab-cyber/hackbot.git"

## 2. Install Security Tools (for Agent Mode)

Skip this cell if you only need Chat or Plan mode.

In [ ]:
# Install common security tools available on Colab (Debian/Ubuntu)
!apt-get update -qq
!apt-get install -y -qq nmap nikto dnsutils whois curl wget netcat-openbsd traceroute \
    sslscan dirb hydra john 2>/dev/null
print("\n✅ Security tools installed.")

## 3. Configure AI Provider

Set your API key and provider. Supports: `openai`, `anthropic`, `gemini`, `groq`, `mistral`, `deepseek`, `together`, `openrouter`.

**Free / cheap options:**
- **Groq** — free tier with fast inference ([console.groq.com](https://console.groq.com))
- **Google Gemini** — free tier ([aistudio.google.com](https://aistudio.google.com))
- **OpenRouter** — pay-per-token, many models ([openrouter.ai](https://openrouter.ai))

In [ ]:
import os
from google.colab import userdata

# ──────────────────────────────────────────────────────────
# Option A: Use Colab Secrets (recommended)
#   Go to 🔑 (key icon) in left sidebar → add a secret
#   named HACKBOT_API_KEY with your API key.
# ──────────────────────────────────────────────────────────
try:
    os.environ["HACKBOT_API_KEY"] = userdata.get("HACKBOT_API_KEY")
    print("✅ API key loaded from Colab Secrets.")
except Exception:
    pass

# ──────────────────────────────────────────────────────────
# Option B: Paste your key directly (less secure)
# Uncomment the line below and replace with your actual key.
# ──────────────────────────────────────────────────────────
# os.environ["HACKBOT_API_KEY"] = "your-api-key-here"

# ── Choose provider and model ────────────────────────────
os.environ["HACKBOT_PROVIDER"] = "groq"                         # Change as needed
os.environ["HACKBOT_MODEL"]    = "llama-3.3-70b-versatile"      # Change as needed

print(f"Provider : {os.environ.get('HACKBOT_PROVIDER')}")
print(f"Model    : {os.environ.get('HACKBOT_MODEL')}")
print(f"Key set  : {'Yes' if os.environ.get('HACKBOT_API_KEY') else 'No ⚠️'}")

## 4. Initialize HackBot Engine

In [ ]:
from hackbot.config import load_config, HackBotConfig
from hackbot.core.engine import AIEngine, PROVIDERS
from hackbot.modes.chat import ChatMode
from hackbot.modes.agent import AgentMode
from hackbot.modes.plan import PlanMode
from IPython.display import display, Markdown

config = load_config()
engine = AIEngine(config.ai)

# Validate connection
result = engine.validate_api_key()
status = "✅" if result["valid"] else "❌"
print(f"{status} {result['message']}")

---
## 📟 Chat Mode

Ask any cybersecurity question — vulnerability analysis, tool usage, exploit techniques, compliance, etc.

In [ ]:
chat = ChatMode(engine, config)

def ask(question: str):
    """Send a question to HackBot Chat and render the response."""
    response = chat.ask(question, stream=False)
    display(Markdown(response))
    return response

In [ ]:
# Example: Ask a cybersecurity question
ask("Explain the top 5 OWASP vulnerabilities with examples and mitigation strategies.")

In [ ]:
# Follow-up (conversation history is preserved)
ask("Now show me how to test for SQL injection using sqlmap with practical examples.")

In [ ]:
# Your own question — edit and run
ask("Your question here")

---
## 🕵️ Agent Mode

Autonomous penetration testing. The agent plans, executes tools, analyzes results, and reports findings.

> ⚠️ **Only test targets you have explicit authorization to test.**

In [ ]:
from hackbot.core.runner import ToolRunner

# Check which security tools are available on this Colab instance
runner = ToolRunner(
    allowed_tools=config.agent.allowed_tools,
    timeout=config.agent.timeout,
    safe_mode=config.agent.safe_mode,
)
tools = runner.get_available_tools()
installed = [t for t, ok in tools.items() if ok]
missing   = [t for t, ok in tools.items() if not ok]

print(f"✅ Installed ({len(installed)}): {', '.join(installed)}")
print(f"❌ Missing   ({len(missing)}):  {', '.join(missing)}")

In [ ]:
# ── Start an Agent assessment ────────────────────────────
# Replace with YOUR authorized target.
TARGET = "scanme.nmap.org"        # nmap's public test target
SCOPE  = "Port scan and service enumeration only"

output_tokens = []
def on_token(token):
    output_tokens.append(token)

agent = AgentMode(
    engine=engine,
    config=config,
    on_token=on_token,
)

# Start assessment — the agent will return its initial plan
initial_plan = agent.start(TARGET, scope=SCOPE)
display(Markdown(initial_plan))

In [ ]:
# ── Run agent steps (one at a time) ──────────────────────
# Each call executes the next step. The agent runs tools,
# analyzes results, and decides what to do next.

response, is_complete = agent.step()
display(Markdown(response))

if is_complete:
    print("\n✅ Assessment complete.")

In [ ]:
# ── Run multiple steps automatically ─────────────────────
MAX_STEPS = 10

for i in range(MAX_STEPS):
    if not agent.is_running:
        break
    response, is_complete = agent.step()
    print(f"\n{'='*60}")
    print(f"Step {i+1}")
    print(f"{'='*60}")
    display(Markdown(response))
    if is_complete:
        print("\n✅ Assessment complete.")
        break

# Show findings summary
if agent.findings:
    print(f"\n📋 Findings: {len(agent.findings)}")
    for j, f in enumerate(agent.findings, 1):
        print(f"  {j}. [{f.severity.value}] {f.title}")

In [ ]:
# ── Guide the agent with custom input ────────────────────
response, is_complete = agent.step("Now focus on checking for common web vulnerabilities on port 80.")
display(Markdown(response))

---
## 📋 Plan Mode

Generate structured penetration testing plans with specific tools, commands, and timelines.

**Available plan types:**
`web_pentest`, `network_pentest`, `api_pentest`, `cloud_audit`, `ad_pentest`, `mobile_pentest`, `red_team`, `bug_bounty`

In [ ]:
plan = PlanMode(engine, config)

def create_plan(target, plan_type="web_pentest", scope="", constraints=""):
    """Generate a pentest plan and render it."""
    result = plan.create_plan(
        target=target,
        plan_type=plan_type,
        scope=scope,
        constraints=constraints,
    )
    display(Markdown(result))
    return result

In [ ]:
# Web Application Pentest Plan
create_plan(
    target="example.com",
    plan_type="web_pentest",
    scope="Main web application on port 443",
)

In [ ]:
# Network Pentest Plan
create_plan(
    target="10.0.0.0/24",
    plan_type="network_pentest",
    scope="Internal network segment",
)

In [ ]:
# API Security Assessment Plan
create_plan(
    target="api.example.com",
    plan_type="api_pentest",
)

In [ ]:
# Bug Bounty Plan
create_plan(
    target="bugcrowd-target.com",
    plan_type="bug_bounty",
    scope="*.bugcrowd-target.com — web assets only",
    constraints="No DoS, no social engineering",
)

---
## 🔧 Run Local Ollama Model via Colab GPU (Optional)

If you want to run a **local model** using Colab's free GPU, install Ollama in Colab and use it as the provider.

> Change Runtime → **T4 GPU** first.

In [ ]:
# Install Ollama inside Colab
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import subprocess, time

# Start Ollama server in background
proc = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
time.sleep(3)
print(f"✅ Ollama server started (PID: {proc.pid})")

In [ ]:
# Pull a model — pick one based on Colab's resources:
#   T4 GPU (15 GB)  → tinyllama, mistral, deepseek-r1:14b
#   A100 GPU (40 GB) → llama3.1:70b, mixtral

# For T4 (free tier):
!ollama pull tinyllama

# For cybersecurity tasks (needs ~8 GB):
# !ollama pull xploiter/pentester

# For better quality (needs ~6 GB):
# !ollama pull mistral

In [ ]:
# Reconfigure HackBot to use local Ollama
import os
os.environ["HACKBOT_PROVIDER"] = "ollama"
os.environ["HACKBOT_MODEL"]    = "tinyllama"   # or xploiter/pentester, mistral, etc.
os.environ.pop("HACKBOT_API_KEY", None)         # Ollama doesn't need a key

# Reload
from hackbot.config import load_config
from hackbot.core.engine import AIEngine
from hackbot.modes.chat import ChatMode

config = load_config()
engine = AIEngine(config.ai)
chat   = ChatMode(engine, config)

# Test it
resp = chat.ask("What is a reverse shell?", stream=False)
display(Markdown(resp))

---
## 🌐 Launch HackBot GUI in Colab (Optional)

Run the full web GUI and access it via a tunnel.

In [ ]:
# Install GUI dependencies
!pip install -q "hackbot[gui] @ git+https://github.com/yashab-cyber/hackbot.git"

In [ ]:
# Start the HackBot GUI server in the background
import subprocess, time

gui_proc = subprocess.Popen(
    ["hackbot", "gui", "--host", "0.0.0.0", "--port", "1337"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
time.sleep(3)
print(f"✅ HackBot GUI running on port 1337 (PID: {gui_proc.pid})")

In [ ]:
# Expose via Colab's built-in localtunnel
from google.colab.output import eval_js
url = eval_js("google.colab.kernel.proxyPort(1337)")
print(f"\n🌐 Open HackBot GUI: {url}")

---
## 🧹 Cleanup

In [ ]:
# Stop background processes if running
try:
    proc.terminate()
    print("Ollama server stopped.")
except:
    pass

try:
    gui_proc.terminate()
    print("GUI server stopped.")
except:
    pass

print("✅ Done.")

---

**Links:**
- GitHub: [yashab-cyber/hackbot](https://github.com/yashab-cyber/hackbot)
- Manual: [06-ai-providers.md](https://github.com/yashab-cyber/hackbot/blob/main/manual/06-ai-providers.md)
- Ollama Models: [ollama.com/library](https://ollama.com/library)

**Recommended models for Colab free tier (T4 GPU):**
- `tinyllama` — ultra-light, works everywhere
- `xploiter/pentester` — built for ethical hacking
- `mistral` — solid 7B all-rounder